## École Polytechnique de Montréal
## Département Génie Informatique et Génie Logiciel

## INF8460 – Traitement automatique de la langue naturelle - TP1

## Objectifs d'apprentissage: 

•	Savoir accéder à un corpus, le nettoyer et effectuer divers pré-traitements sur les données
•	Savoir effectuer une classification automatique des textes pour l’analyse de sentiments
•	Evaluer l’impact des pré-traitements sur les résultats obtenus


## Équipe et contributions 
Veuillez indiquer la contribution effective de chaque membre de l'équipe en pourcentage et en indiquant les modules ou questions sur lesquelles chaque membre a travaillé


Nom Étudiant 1: Luu Thien-Kim (1834378) 33.33%

Nom Étudiant 2: Mellouk Souhaila (1835144) 33.33%

Nom Étudiant 3: Younes Mourad (1832387) 33.33%

Nous avons tous travaillé ensemble sur chaque question

## Librairies externes

In [165]:
import os
import pandas as pd
from typing import List, Literal, Tuple

## Valeurs globales

In [166]:
data_path = "data"
output_path = "output"

## Données

In [167]:
def read_data(path: str) -> Tuple[List[str], List[bool], List[Literal["M", "W"]]]:
    data = pd.read_csv(path)
    inputs = data["response_text"].tolist()
    labels = (data["sentiment"] == "Positive").tolist()
    gender = data["op_gender"].tolist()
    return inputs, labels, gender

In [168]:
train_data = read_data(os.path.join(data_path, "train.csv"))
test_data = read_data(os.path.join(data_path, "test.csv"))

train_data = ([text.lower() for text in train_data[0]], train_data[1], train_data[2])
test_data = ([text.lower() for text in test_data[0]], test_data[1], test_data[2])

## 1. Pré-traitement et Exploration des données

### Lecture et prétraitement

Dans cette section, vous devez compléter la fonction preprocess_corpus qui doit être appelée sur les fichiers train.csv et test.csv. La fonction preprocess_corpus appellera les différentes fonctions créées ci-dessous. Les différents fichiers de sortie doivent se retrouver dans le répertoire output.  Chacune des sous-questions suivantes devraient être une ou plusieurs fonctions.

In [169]:
train_path = os.path.join(data_path, "train.csv")
test_path = os.path.join(data_path, "test.csv")

train_phrases_path = os.path.join(output_path, "train_phrases.csv")
test_phrases_path = os.path.join(output_path, "test_phrases.csv")

#### 1) Segmentez chaque corpus en phrases, et stockez-les dans un fichier `nomcorpus`_phrases.csv (une phrase par ligne)

In [405]:
import nltk
nltk.download("punkt") 
nltk.download("wordnet")
import csv

def segmentSentences(path) :
    data = read_data(path)
    corpus = data[0]
    if not os.path.isdir(output_path) :
        try:
            os.mkdir(output_path)
        except OSError:
            print ("Creation of the directory %s failed" % path)
        else:
            print ("Successfully created the directory %s " % path)
    newFilePath = output_path + '/' + os.path.splitext(os.path.basename(path))[0] + "_phrases.csv"
    file = open(newFilePath, "w")
    with open(newFilePath, "w") as f: 
        f.write("response_text" + ',' + "sentiment" + ',' + "op_gender" +'\n')
        for i in range(len(corpus)) :
            sentences = nltk.sent_tokenize(corpus[i])
            for sentence in sentences:
                sentence = sentence.replace('"', '""').replace('"', '""')
                f.write('"'+ sentence +'"' + ',' + '"' +str(data[1][i])+ '"'+ ',' + '"'+data[2][i] + '"\n')
                
    return newFilePath


[nltk_data] Downloading package punkt to /Users/kimluu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/kimluu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [406]:
segmentSentences(train_path)
segmentSentences(test_path)

'output/test_phrases.csv'

#### 2) Normalisez chaque corpus au moyen d’expressions régulières en annotant les négations avec _Neg L’annotation de la négation doit ajouter un suffixe _NEG à chaque mot qui apparait entre une négation et un signe de ponctuation qui identifie une clause. Exemple : 
No one enjoys it.  no one_NEG enjoys_NEG it_NEG .

I don’t think I will enjoy it, but I might.  i don’t think_NEG i_NEG will_NEG enjoy_NEG it_NEG, but i might.

In [407]:
def getPath(path) :
    if "train" in path :
        path = train_path
    elif "test" in path :
        path = test_path
        
    return path

In [408]:
import re

def normalize(path) :
    with open(path, "r") as f :
        data = list(f)
    
    newFilePath = output_path + '/' + os.path.splitext(os.path.basename(getPath(path)))[0] + "_negation.csv"
    file = open(newFilePath, "w")
    with open(newFilePath, "w") as f:
        for sentence in data:
            match = re.sub("(?i)(?<=not |n't | no )(.*?[,.(?!;]+)", lambda m: m.group(1).replace(" ", "_NEG ")
                           .replace(".", "_NEG.").replace(",", "_NEG,").replace("?", "_NEG?").replace("!", "_NEG!")
                           .replace("(", "_NEG(").replace(";", "_NEG;"), sentence)
            f.write(match)
            
    return newFilePath
            

In [409]:
normalize(train_phrases_path)
normalize(test_phrases_path)

'output/test_negation.csv'

#### 3) Segmentez chaque phrase en mots (tokenisation) et stockez-les dans un fichier `nomcorpus`_mots.csv. (Une phrase par ligne, chaque token séparé par un espace, il n’est pas nécessaire de stocker la phrase non segmentée ici) ;

In [431]:
def tokenize(path) :
    sentences = []
    
#     with open(path, "r") as f :
#         data = list(f)
    data = read_data(path)
    corpus = data[0]        
#     print(data[1])
    newFilePath = output_path + '/' + os.path.splitext(os.path.basename(getPath(path)))[0] + "_mots.csv"
    file = open(newFilePath, "w")
    with open(newFilePath, "w") as f: 
        f.write("response_text" + ',' + "sentiment" + ',' + "op_gender" +'\n')
        for i in range(len(corpus)) :
            listTokens = nltk.word_tokenize(corpus[i])
            tokens = ' '.join(listTokens)
            print(tokens)
            tokens = sentence.replace('"', '""').replace('"', '')
            f.write('"' + tokens + '"' + ',' + '"' + str(data[1][i])+ ',' + data[2][i] + '"\n')
                
    return newFilePath

In [432]:
# tokenize(train_phrases_path)
tokenize(test_phrases_path)

# train_mots_path = os.path.join(output_path, "train_mots.csv")
# test_mots_path = os.path.join(output_path, "test_mots.csv")

Thanks back !
Yep , University of Alberta .
You live around here ?
please do n't sell my land Steve
just shaking my head at the ignorance and deliberate ignoring of the facts about FDR , Pearl Harbor , and WWII .
To be contemplated during your tri , perhaps ?
Pshh ... Is that how you treat my props .. Just go around deleting them ? ! ? !
Sureeeeeeeeeeeeeeeeeeee I see how it is .
: pYeah there 's definitely still some bugs around here .
My workout from last night posted with today 's date on it .
lol
Thanks !
I also love bacon .
: )
Hello Isaac !
My copy arrived yesterday in France , I 'm so happy and very excited to read it ! ! !
XD
We need to keep Bob Menendez in congress .
It 's really an excellent lecture , I believe in her.so , fake it till you make it !
I 'm human according to most of the questions .
And his tone was great , I had a lot of fun : )
You 're both awesome ! !
!
B Mattek is awesome , I love her bad girl risque style .
Brit : I 'm so glad u r on Fox tonight .
You calm p

Voy a estar haciendo el LA marathon .
Se miran bellos los 3 .
Every Democrat in Congress conspired to committ FRAUD upon the American Citizens PERIOD !
!
No smart phone means no updating as I go .
: (
You hit a sensible snare Ron , thank you so much for doing and sharing this .. To me this is another eye opener in the direction of that big problems are well served with simple solutions ..
Solutions so simple and profound its almost unimaginable .
Senator Menendez I thank you for all your hard work .
You have my VOTE .
http : //twitchy.com/2015/05/10/watch-mark-halperins-ghastly-cuban-authenticity-interrogation-of-ted-cruz-video/
Hi Richard , from the Neal 's in Florida , same to you and your family !
Have you lost your mind ! ! ! ? ? ?
Obama has done nothing impeachable ? ?
Exactly WHICH constitution have you been reading ? ?
?
Blinded by bias ... not even close to reality .
What a disturbing statement for a US Congressman to post on FB .
If you think this is true you havent explored D

Can anybody recommend good books on the topics or training programs / courses one can take ?
Thanks , Dieter
Mind-blowing material , excellently delivered !
Thank you
Haha .... Thank-you !
: )
This is a great start .
I can not wait to see what you show us next .
Things happen before we know it just work on things and your life and body will improve with time
Yes I have the same .
I can put a sheet on in a few seconds .
I change the sheet regularly so maybe Ive just got good at it .
I just wanted to thank you for voting No on HR 1599. even though it passed , 80 % of Americans disagree with it .
you represent the people and thats a great thing
Your vag has its own exhibit .
laser guarded so it wont get bombarded with solicitation .
Woot Awkwafinas in a museum this calls for celebration
We can all try , but whores are as infinite as the universe is big .
Most will get away .
He described his different-point of view beautifully .
I hope to see more talks from him .
I 'd wish I could study 

'output/test_mots.csv'

#### 4) Lemmatisez les mots et stockez les lemmes dans un fichier `nomcorpus`_lemmes.csv (une phrase par ligne, les lemmes séparés par un espace) ;

In [177]:
def lemmatize(path) :
    with open(path, "r") as f :
        data = list(f)
        
    newFilePath = output_path + '/' + os.path.splitext(os.path.basename(getPath(path)))[0] + "_lemmes.csv"
    lemmzer = nltk.WordNetLemmatizer()
    
    file = open(newFilePath, "w")
    with open(newFilePath, "w") as f: 
        for sentences in data :
            lemmes = [lemmzer.lemmatize(token) for token in sentences.split()]
            sentences = ' '.join(lemmes)
            f.write(sentences+'\n')
                
    return newFilePath

In [178]:
lemmatize(train_mots_path)
lemmatize(test_mots_path)

'output/test_lemmes.csv'

#### 5) Retrouvez la racine des mots (stemming) en utilisant nltk.PorterStemmer(). Stockez-les dans un fichier `nomcorpus`_stems.csv (une phrase par ligne, les racines séparées par une espace) ;

In [179]:
def stemmize(path) :    
    with open(path, "r") as f :
        reader = csv.reader(f)
        data = list(reader)
        
    path = getPath(path)
    newFilePath = output_path + '/' + os.path.splitext(os.path.basename(path))[0] + "_stems.csv"
    
    stemmer = nltk.PorterStemmer()
    
    file = open(newFilePath, "w")
    with open(newFilePath, "w") as f: 
        for sentences in data :
            for sentence in sentences :
                stems = [stemmer.stem(token) for token in sentence.split()]
                sentences = ' '.join(stems)
                f.write('"' + sentences + '"\n')
                
    return newFilePath
    

In [180]:
stemmize(train_mots_path)
stemmize(test_mots_path)

'output/test_stems.csv'

#### 6) Ecrivez une fonction qui supprime les mots outils (stopwords) du corpus. Vous devez utiliser la liste de stopwords de NLTK ;

In [181]:
nltk.download("stopwords")
from nltk.corpus import stopwords
stopwords.words("english")

def deleteStopWords(path) :
    with open(path, "r") as f :
        reader = csv.reader(f)
        data = list(reader)
        
    path = getPath(path)
    newFilePath = output_path + '/' + os.path.splitext(os.path.basename(path))[0] + "_stopWords.csv"
    stopwords_english = set(stopwords.words("english"))
    output = []
    
    file = open(newFilePath, "w")
    with open(newFilePath, "w") as f: 
        for sentences in data :
            for sentence in sentences :
                newSentence = [token for token in nltk.word_tokenize(sentence) if token not in stopwords_english]
                sentences = ' '.join(newSentence)
                output.append(sentences)
                f.write('"' + sentences + '"\n')
                
    return output
                
#enlever la création de nouveaux fichiers
    

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kimluu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [182]:
deleteStopWords(train_mots_path)
deleteStopWords(test_mots_path)

['Thanks back !',
 'Yep , University Alberta .',
 'You live around ?',
 "please n't sell land Steve",
 'shaking head ignorance deliberate ignoring facts FDR , Pearl Harbor , WWII .',
 'To contemplated tri , perhaps ?',
 'Pshh ... Is treat props .. Just go around deleting ? ! ? !',
 'Sureeeeeeeeeeeeeeeeeeee I see .',
 ": pYeah 's definitely still bugs around .",
 "My workout last night posted today 's date .",
 'lol',
 'Thanks !',
 'I also love bacon .',
 ': )',
 'Hello Isaac !',
 "My copy arrived yesterday France , I 'm happy excited read ! ! !",
 'XD',
 'We need keep Bob Menendez congress .',
 "It 's really excellent lecture , I believe her.so , fake till make !",
 "I 'm human according questions .",
 'And tone great , I lot fun : )',
 "You 're awesome ! !",
 '!',
 'B Mattek awesome , I love bad girl risque style .',
 "Brit : I 'm glad u r Fox tonight .",
 'You calm people bring common sense .',
 'You class act !',
 '!',
 'Good luck .',
 'I really hope make .',
 'Please reach Mr. Trum

#### 7) Écrivez une fonction preprocess_corpus(corpus) qui prend un corpus brut stocké dans un fichier.csv, effectue les étapes précédentes, puis stocke le résultat de ces différentes opérations dans un fichier corpus _norm.csv

In [183]:
def preprocess_corpus(input_file: str, output_file: str) :
    #to do : vérifier si c'est bien le résultat voulu
    results = deleteStopWords(stemmize(lemmatize(tokenize(normalize(segmentSentences(input_file))))))
    file = open(output_file, "w")
    with open(output_file, "w") as f:
        for element in results :
            r = element.replace('"', '""').replace('"', '')
            f.write('"' + r + '"\n')
            
    

In [184]:
preprocess_corpus(
   os.path.join(data_path, "train.csv"), os.path.join(output_path, "train_norm.csv")
)
preprocess_corpus(
   os.path.join(data_path, "test.csv"), os.path.join(output_path, "test_norm.csv")
)

### Exploration des données

#### 1)

Complétez les fonctions retournant les informations suivantes (une fonction par information, chaque fonction prenant en argument un corpus composé d'une liste de phrases segmentées en tokens(tokenization)) ou une liste de genres et une liste de sentiments:

In [185]:
corpus = [['soso.', 'kim a acheté un mbp13 silver!', 'mourad a acheté un mbp16 spacegrey']]
corpus = [['soso', 'a', 'acheté', 'un', 'mbp16', 'silver', '.', 'Je', 'ne', 'suis', 'pas', 'daccord!'],
          ['kim', 'a', 'acheté', 'un', 'mpb13', 'silver','.'], 
          ['mourad', 'a', 'acheté', 'un', 'mbp16', 'spacegrey', '.', 'Quil', 'aime', 'beaucoup']]
corpus = [["I", "do", "not", "agree_NEG", "with_NEG", "this_NEG!", "I", "prefer", "the", "other", "option"],
           ["I", "really", "like", "that", "new", "mbp16", "Mourad", "made", "a", "good", "choice"],
           ["I", "don't", "think_NEG", "that_NEG.", "I", "prefer", "the", "mbp13"]]

In [186]:
#Return if the corpus is tokenized or not
def isTokenized(corpus) :
    for sentences in corpus :
        for sentence in corpus :
            for l in sentence :
                if ' ' in l :
                    return 0
    return 1

In [187]:
#Return the corpus as a list of documents, that are not tokenizated, but segmented in sentences
def getListOfSentences(corpus):
    listOfDocs = []
    listOfTokens = []
    if isTokenized(corpus) :
        for sentences in corpus :
            for token in sentences :
                listOfTokens.append(token)
            s = ' '.join(listOfTokens)
            s = nltk.sent_tokenize(s)
            listOfDocs.append(s)
            listOfTokens = []
    return listOfDocs

In [188]:
getListOfSentences(corpus)

[['I do not agree_NEG with_NEG this_NEG!', 'I prefer the other option'],
 ['I really like that new mbp16 Mourad made a good choice'],
 ["I don't think_NEG that_NEG.", 'I prefer the mbp13']]

##### a. Le nombre total de tokens (mots non distincts)

In [189]:
def getNumberOfTokens(corpus):
    corpus = getListOfSentences(corpus)
    count = 0
    for sentences in corpus :
        for sentence in sentences :
            count = count + len(nltk.word_tokenize(sentence))
    return count

In [190]:
getNumberOfTokens(corpus)

33

##### b. Le nombre total de types

In [191]:
def getNumberOfTypes(corpus):
    corpus = getListOfSentences(corpus)
    listOfTokens = []
    for sentences in corpus :
        for sentence in sentences :
            tokenList = nltk.word_tokenize(sentence)
            for token in tokenList :
                listOfTokens.append(token)
    listOfTypes = list(dict.fromkeys(listOfTokens))  
    return len(listOfTypes)

In [192]:
getNumberOfTypes(corpus)

26

##### c. Le nombre total de phrases avec négation

In [193]:
def getNumberOfNeg(corpus) :
    corpus = getListOfSentences(corpus)
    numberOfNegativeSentences = 0;
    for sentences in corpus:
        for sentence in sentences:
            if "_NEG" in sentence:
                numberOfNegativeSentences = numberOfNegativeSentences + 1
    return numberOfNegativeSentences

In [194]:
getNumberOfNeg(corpus)

2

##### d. Le ratio token/type

In [195]:
def getRatioTokenType(corpus):
    return float(getNumberOfTokens(corpus)/getNumberOfTypes(corpus))

In [196]:
getRatioTokenType(corpus)

1.2692307692307692

##### e. Le nombre total de lemmes distincts

In [197]:
import nltk
def getLemmesNumber(corpus):
    corpus = getListOfSentences(corpus)
    lemmzer = nltk.WordNetLemmatizer()
    lemmesList = []
    for sentences in corpus :
        for sentence in sentences :
            lemmes = [lemmzer.lemmatize(token) for token in sentence.split()]
            for lemme in lemmes :  
                lemmesList.append(lemme)
    
    lemmesList = list(dict.fromkeys(lemmesList))   
    return len(lemmesList)

In [198]:
getLemmesNumber(corpus)

24

##### f. Le nombre total de racines (stems) distinctes

In [199]:
import nltk
def getStemsNumber(corpus):
    corpus = getListOfSentences(corpus)
    stemmer = nltk.PorterStemmer()
    stemsList = []
    for sentences in corpus :
        for sentence in sentences :
            stems = [stemmer.stem(token) for token in sentence.split()]
            for stem in stems :
                stemsList.append(stem)
    stemsList = list(dict.fromkeys(stemsList))     
    return len(stemsList)

In [200]:
getStemsNumber(corpus)

24

##### g. Le nombre total de documents (par classe)

In [201]:
def getNumberOfDocPerClass(sentiments):
    countPositive = 0
    countNegative = 0
    for sentiment in sentiments : 
        if sentiment : #positif
            countPositive = countPositive + 1
        else : #negatif
            countNegative = countNegative + 1
    return countPositive, countNegative
        
        

In [202]:
semtiments = [0,
          0,
          0,
          0,
          0, #5 negatif
          1,
          1,
          1,
          1,
          1,
          1] #6 positif

getNumberOfDocPerClass(semtiments)

(6, 5)

##### h. Le nombre total de phrases (par classe)

In [203]:
def getNumberOfSentencesPerClass(corpus, sentiments) :
    corpus = getListOfSentences(corpus)
    countSentencesPositives = 0
    countSentencesNegatives = 0
    for i in range(len(corpus)):
        if sentiments[i] : #positive
            countSentencesPositives = countSentencesPositives + len(corpus[i])
        else : #negative
            countSentencesNegatives = countSentencesNegatives + len(corpus[i])          
    return countSentencesPositives, countSentencesNegatives   

In [204]:
sentiments = [0,1,1]
getNumberOfSentencesPerClass(corpus, sentiments)

(3, 2)

##### i. Le nombre total de phrases avec négation (par classe)

In [205]:
def getNumberOfNegativeSentences(corpus, sentiments) :
    corpus = getListOfSentences(corpus)
    countNegativeSentencesPositives = 0
    countNegativeSentencesNegatives = 0
    i = 0;
    for sentences in corpus:
        for sentence in sentences:
            if sentiments[i]: #positive
                if "_NEG" in sentence:
                    countNegativeSentencesPositives = countNegativeSentencesPositives + 1
            else: #negative
                if "_NEG" in sentence:
                    countNegativeSentencesNegatives = countNegativeSentencesNegatives + 1
            i = i + 1
    return countNegativeSentencesPositives, countNegativeSentencesNegatives

In [206]:
sentiments = [0,1,1,1,1]
getNumberOfNegativeSentences(corpus, sentiments)

(1, 1)

##### j. Le pourcentage de réponses positives par genre de la personne à qui cette réponse est faite (op_gender)

In [207]:
genders = [['M', 'M', 'W', 'W', 'M', 'M', 'W', 'W', 'M', 'M', 'W', 'W']]
sentiments = [1 , 0, 1, 0, 1 , 0, 1,
             0, 0, 1,  0, 0]

In [208]:
def getPourcentageOfPositiveReponsesPerGender(genders, sentiments):
    countPosM = countPosW = 0
    iterator = 0
    totalResponse = len(sentiments)
  
    for sentiment in sentiments:
        if sentiment :
            if genders[0][iterator] == 'M':
                countPosM = countPosM + 1
            elif genders[0][iterator] == 'W':
                countPosW = countPosW + 1
        iterator = iterator + 1
  
    pourcentageW = float(countPosW / totalResponse) * 100
    pourcentageM = float(countPosM / totalResponse) * 100
  
    return(pourcentageM,pourcentageW)       

In [209]:
getPourcentageOfPositiveReponsesPerGender(genders, sentiments)

(25.0, 16.666666666666664)

#### 2) Écrivez la fonction explore(corpus, sentiments, genders) qui calcule et affiche toutes ces informations, précédées d'une légende reprenant l’énoncé de chaque question (a,b, ….j).

In [210]:
def explore(
    corpus: List[List[str]], sentiments: List[bool], genders: List[Literal["M", "W"]]
) -> None:
    print("Le nombre total de tokens (mots non distincts) : " + getNumberOfTokens(corpus) + "\n")
    print("Le nombre total de types : " + getNumberOfTypes(corpus) + "\n")
    print("Le nombre total de phrases avec négation : " + getNumberOfNeg(corpus)  + "\n")
    print("Le ratio token/type : " + getRatioTokenType(corpus) + "\n")
    print("Le nombre total de lemmes distincts : " + getLemmesNumber(corpus) + "\n")
    print("Le nombre total de racines (stems) distinctes : " + getStemsNumber(corpus) + "\n")
    print("Le nombre total de documents (par classe) : " + getNumberOfDocPerClass(semtiments) + "\n")
    print("Le nombre total de phrases (par classe) : " + getNumberOfSentencesPerClass(corpus, sentiments) + "\n")
    print("Le nombre total de phrases avec négation (par classe) : " + getNumberOfNegativeSentences(corpus, sentiments)  + "\n")
    print("Le pourcentage de réponses positives par genre de la personne à qui cette réponse est faite (op_gender) : " 
          + getPourcentageOfPositiveReponsesPerGender(sentiments, genders)  + "\n")

    

#### 3) Calculer une table de fréquence (lemme, rang (le mot le plus fréquent a le rang 1 etc.) ; fréquence (le nombre de fois où il a été vu dans le corpus).  Seuls les N mots les plus fréquents du vocabulaire (N est un paramètre) doivent être gardés. Vous devez stocker les 1000 premières lignes de cette table dans un fichier nommé table_freq.csv

In [211]:
def calculateFrequences(corpus) :
   
    corpus = getListOfSentences(corpus)
    lemmzer = nltk.WordNetLemmatizer()
    lemmesList = []
    sorted_dict = {}
   
    for sentences in corpus :
        for sentence in sentences :
            lemmes = [lemmzer.lemmatize(token) for token in sentence.split()]
            for lemme in lemmes :
                lemmesList.append(lemme)
               
    for word in lemmesList:
        if word not in sorted_dict:
            sorted_dict[word] = 0
        sorted_dict[word] += 1
    words = sorted_dict.items()
    sorted_lemme = sorted(words, key= lambda kv: kv[1], reverse=True)
    return sorted_lemme

In [212]:
calculateFrequences(corpus)

[('I', 5),
 ('prefer', 2),
 ('the', 2),
 ('do', 1),
 ('not', 1),
 ('agree_NEG', 1),
 ('with_NEG', 1),
 ('this_NEG!', 1),
 ('other', 1),
 ('option', 1),
 ('really', 1),
 ('like', 1),
 ('that', 1),
 ('new', 1),
 ('mbp16', 1),
 ('Mourad', 1),
 ('made', 1),
 ('a', 1),
 ('good', 1),
 ('choice', 1),
 ("don't", 1),
 ('think_NEG', 1),
 ('that_NEG.', 1),
 ('mbp13', 1)]

## 2. Classification automatique

### a) Classification  automatique avec un modèle sac de mots (unigrammes), Naive Bayes et la régression logistique

En utilisant la librairie scikitLearn et l’algorithme Multinomial Naive Bayes et Logistic Regression, effectuez la classification des textes avec un modèle sac de mots unigramme pondéré avec TF-IDF.  Vous devez entrainer chaque modèle sur l’ensemble d’entrainement et le construire à partir de votre fichier corpus_train.csv. 

Construisez et sauvegardez votre modèle sac de mots avec les données d’entrainement en testant les pré-traitements suivants (séparément et en combinaison): tokenisation, lemmatisation, stemming, normalisation des négations, et suppression des mots outils. Vous ne devez garder que la combinaison d’opérations qui vous donne les meilleures performances sur le corpus de test. Indiquez dans un commentaire les pré-traitements qui vous amènent à votre meilleure performance (voir la section 3 – évaluation). Il est possible que la combinaison optimale ne soit pas la même selon que vous utilisiez la régression logistique ou Naive Bayes. On s’attend à avoir deux modèles optimaux, un pour Naive Bayes, et un avec régression logistique.

In [216]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

In [316]:
def getDataSet(train_csv, test_csv):
    trainPath = os.path.join(output_path, train_csv)
    testPath = os.path.join(output_path, test_csv)
    
    trainData_ = []
    testData_ = []
    
    with open(trainPath, "r") as f :
        trainData = list(f)
     
    for data in trainData :
        data = data.replace('"', '').replace('\n', '')
        trainData_.append(data)
    
    with open(testPath, "r") as f :
        testData = list(f)
        
    for data in testData :
        data = data.replace('"', '').replace('\n', '')
        testData_.append(data)
#     print(testData_)
    
#     trainData = pd.read_csv(trainPath)
#     testData = pd.read_csv(testPath)
#     print(testData[0])
    
#     trainInputs = trainData
#     testInputs = testData
    
    training_data = (trainData_, train_data[1], train_data[2])
    testing_data = (testData_, test_data[1], test_data[2])
    return training_data, testing_data

In [317]:
training_data, testing_data = getDataSet("train_phrases.csv", "test_phrases.csv" )
# print(training_data, testing_data)

### Naive Bayes

In [311]:
def naiveBayes(train_data, test_data):
    vectorizer = TfidfVectorizer()    
    vectors = vectorizer.fit_transform(train_data[0])
    clf = MultinomialNB(alpha=0.5)
    clf.fit(vectors, train_data[1])
    
    vectors_test = vectorizer.transform(test_data)
    y_pred = clf.predict(vectors_test)
    return y_pred

### Régression Logistique

In [312]:
def logisticsRegression(train_data, test_data):
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform(train_data[0])
    model = LogisticRegression(C=1.0)
    model.fit(vectors, train_data[1])
    
    vectors_test = vectorizer.transform(test_data[0])
    y_pred = model.predict(vectors_test)
    return y_pred

###  b) Autre représentation pour l’analyse de sentiments et classification automatique

On vous propose maintenant d’utiliser une nouvelle représentation de chaque document à classifier.
Vous devez créer à partir de votre corpus la table suivante :

| Vocabulaire | Freq-positive | Freq-négative |
|-------------|---------------|---------------|
| happy | 10 | 1 |
| ... | ... | ... |

Où :

• Vocabulaire représente tous les types (mots uniques) de votre corpus d’entrainement

• Freq-positive : représente la somme des fréquences du mot dans tous les documents de la classe positive

• Freq-négative : représente la somme des fréquences du mot dans tous les documents de la classe négative

Notez qu’en Python, vous pouvez créer un dictionnaire associant à tout (mot, classe) une fréquence.
Ensuite il vous suffit de représenter chaque document par un vecteur à 3 dimensions dont le premier élément représente un biais (initialisé à 1), le deuxième élément représente la somme des fréquences positives (freq-pos) de tous les mots uniques (types) du document et enfin le troisième élément représente la somme des fréquences négative (freq-neg) de tous les mots uniques du document. 

En utilisant cette représentation ainsi que les pré-traitements suggérés, trouvez le meilleur modèle possible en testant la régression logistique et Naive Bayes. Vous ne devez fournir que le code de votre meilleur modèle dans votre notebook.

In [340]:
import pandas as pd
corpus = test_data
voc = []
freq_pos = []
freq_neg = []

dict = {
  "mot": "",
  "classe": 0
}

dictionnaries = []

listOfTokens = []

# for sentences in corpus[0] :       
#     tokenList = nltk.word_tokenize(sentences)
#     for token in tokenList :
#         listOfTokens.append(token)
# voc = list(dict.fromkeys(listOfTokens))
# print(len(voc))

for i in range(len(corpus[0])) :
    tokenList = nltk.word_tokenize(corpus[0][i])
    for token in tokenList :
        listOfTokens.append(token)
        dic = {"mot" : token, "classe": corpus[1][i]}
#         dictionnaries = 
        dictionnaries.append((token,corpus[1][i]))
#     listOfTokens = list(dict.fromkeys(listOfTokens))

print(dictionnaries)
    
# for i in range(len(dictionnaries)):
#     dictionnaries = list(dict.fromkeys(dictionnaries))



# for i in voc :
#     dictionnaries.append(dict = {"mot": token , "classe": 0})
    

# for sentences in corpus[1] :
    

# for sentences in corpus[2] :
    
    
# data = [['happy', 2, 3], [3, 4, 5]]
pd.DataFrame(data, columns=["Vocabulaire", "Freq-positive", "Freq-négative"])

[('thanks', True), ('back', True), ('!', True), ('yep', True), (',', True), ('university', True), ('of', True), ('alberta', True), ('.', True), ('you', True), ('live', True), ('around', True), ('here', True), ('?', True), ('please', False), ('do', False), ("n't", False), ('sell', False), ('my', False), ('land', False), ('steve', False), ('just', False), ('shaking', False), ('my', False), ('head', False), ('at', False), ('the', False), ('ignorance', False), ('and', False), ('deliberate', False), ('ignoring', False), ('of', False), ('the', False), ('facts', False), ('about', False), ('fdr', False), (',', False), ('pearl', False), ('harbor', False), (',', False), ('and', False), ('wwii', False), ('.', False), ('to', True), ('be', True), ('contemplated', True), ('during', True), ('your', True), ('tri', True), (',', True), ('perhaps', True), ('?', True), ('pshh', True), ('...', True), ('is', True), ('that', True), ('how', True), ('you', True), ('treat', True), ('my', True), ('props', True),

,Vocabulaire,Freq-positive,Freq-négative
0,happy,2,3
1,3,4,5


## 3. Évaluation et discussion

#### a) Pour déterminer la performance de vos modèles, vous devez tester vos modèles de classification sur l’ensemble de test et générer vos résultats pour chaque modèle dans une table avec les métriques suivantes : Accuracy et pour chaque classe, la précision, le rappel et le F1 score. On doit voir cette table générée dans votre notebook avec la liste de vos modèles de la section 2 et leurs performances respectives. 

In [313]:
y_pred_bayes = naiveBayes(training_data, testing_data)
# print(classification_report(test_data[1], y_pred_bayes))

ValueError: Found input variables with inconsistent numbers of samples: [18399, 9041]

In [284]:
y_pred_regression = logisticsRegression(train_data, test_data)
print(classification_report(test_data[1], y_pred_regression))

              precision    recall  f1-score   support

       False       0.80      0.53      0.64       254
        True       0.86      0.95      0.90       751

    accuracy                           0.85      1005
   macro avg       0.83      0.74      0.77      1005
weighted avg       0.84      0.85      0.84      1005



#### b) Générez un graphique qui représente la performance moyenne (mean accuracy – 10 Fold cross-validation) de vos différents modèles par tranches de 500 textes sur l’ensemble d’entrainement.

#### c) Que se passe-t-il lorsque le paramètre de régularisation de la régression logisque (C) est augmenté ?

## 4. Analyse et discussion

#### a) En considérant les deux types de représentations, répondez aux question suivantes en reportant la question dans le notebook et en inscrivant votre réponse:

#### b) Quel est l’impact de l’annotation de la négation ?

#### c) La suppression des stopwords est-elle une bonne idée pour l’analyse de sentiments ?

#### d) Le stemming et/ou la lemmatisation sont-ils souhaitables dans le cadre de l’analyse de sentiments ?

## 5. Contribution

Complétez la section en haut du notebook indiquant la contribution de chaque membre de l’équipe en indiquant ce qui a été effectué par chaque membre et le pourcentage d’effort du membre dans le TP. 